Частина 2


In [2]:
import pandas as pd
import numpy as np
import timeit

In [3]:
# Читання з урахуванням розділювача і пропусків
# Вказуємо шлях до файлу
file_path = "household_power_consumption.txt"

# Читаємо дані
df = pd.read_csv(file_path, sep=';', low_memory=False)
print(df.head())
print(df.info())
print(df.describe())

         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column      

In [4]:
# Тепер можна замінювати '?' на np.nan
df.replace('?', np.nan, inplace=True)

# Конвертуємо числові колонки
numeric_cols = ['Global_active_power', 'Global_reactive_power', 'Voltage', 
                'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Перевіримо пропущені значення
print(df.isnull().sum())

Date                         0
Time                         0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


In [16]:
# Заповнимо пропуски середнім значенням
df.fillna(df.mean(), inplace=True)

In [5]:
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df.set_index('Datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)

In [6]:
def clean_data(df):
    df_clean = df.copy()
    df_clean.replace('?', np.nan, inplace=True)
    for col in numeric_cols:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    df_clean.fillna(df_clean.mean(), inplace=True)
    return df_clean

# Профілювання часу
execution_time = timeit.timeit(lambda: clean_data(df), number=10)
print(f"Середній час виконання: {execution_time/10:.5f} секунд")

Середній час виконання: 0.47377 секунд


In [7]:
def filter_active_power(df, threshold=5):
    """Обрати записи, де Global_active_power перевищує threshold кВт"""
    return df[df['Global_active_power'] > threshold]


In [8]:
def filter_current_and_compare(df, current_min=19, current_max=20):
    subset = df[(df['Global_intensity'] >= current_min) & (df['Global_intensity'] <= current_max)].copy()
    subset['group1'] = subset['Sub_metering_1'] + subset['Sub_metering_2']
    subset['group2'] = subset['Sub_metering_3'] + subset['Global_reactive_power']
    return subset[subset['group1'] > subset['group2']]

In [9]:
def random_sample_means(df, n=500000):
    sample = df.sample(n=n, replace=False, random_state=42)
    sample['group1'] = sample['Sub_metering_1']
    sample['group2'] = sample['Sub_metering_2']
    sample['group3'] = sample['Sub_metering_3']
    means = sample[['group1','group2','group3']].mean()
    return sample, means

In [10]:
def evening_selection(df):
    # створюємо колонку з часом
    df['hour'] = df.index.hour
    subset = df[(df['hour'] >= 18) & (df['Global_active_power'] > 6)]
    
    # групи споживання
    subset['group1'] = subset['Sub_metering_1']  # пральна машина
    subset['group2'] = subset['Sub_metering_2']  # холодильник/освітлення
    subset['group3'] = subset['Sub_metering_3']  # інші

    # вибираємо ті, де група2 найбільша
    subset = subset[(subset['group2'] > subset['group1']) & (subset['group2'] > subset['group3'])]

    # ділимо на дві половини
    half = len(subset)//2
    first_half = subset.iloc[:half:3]   # кожен третій
    second_half = subset.iloc[half::4]  # кожен четвертий

    return pd.concat([first_half, second_half])

In [11]:
def normalize_and_standardize(df, cols):
    # Нормалізація [0,1]
    for col in cols:
        df[col+'_norm'] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
    # Стандартизація (z-score)
    for col in cols:
        df[col+'_std'] = (df[col] - df[col].mean()) / df[col].std()
    
    return df

In [12]:
def correlation(df, col1, col2):
    # Пірсон (як раніше)
    pearson = df[col1].corr(df[col2], method='pearson')
    
    # Спірмен (через ранги)
    rank1 = df[col1].rank()
    rank2 = df[col2].rank()
    spearman = rank1.corr(rank2)  # кореляція Пірсона між рангами
    return pearson, spearman

In [17]:
def one_hot_encode(df, col):
    return pd.get_dummies(df, columns=[col], prefix=[col])

In [18]:
df_filtered = filter_active_power(df)
print(df_filtered)

                     Global_active_power  Global_reactive_power  Voltage  \
Datetime                                                                   
2006-12-16 17:25:00                5.360                  0.436   233.63   
2006-12-16 17:26:00                5.374                  0.498   233.29   
2006-12-16 17:27:00                5.388                  0.502   233.74   
2006-12-16 17:35:00                5.412                  0.470   232.78   
2006-12-16 17:36:00                5.224                  0.478   232.99   
...                                  ...                    ...      ...   
2010-11-22 18:40:00                5.408                  0.150   231.50   
2010-11-22 18:41:00                5.528                  0.144   232.48   
2010-11-24 07:50:00                5.172                  0.050   235.18   
2010-11-24 07:51:00                5.750                  0.000   234.40   
2010-11-25 07:21:00                5.074                  0.240   238.55   

           

In [19]:
df_current = filter_current_and_compare(df)
print(df_current)


                     Global_active_power  Global_reactive_power  Voltage  \
Datetime                                                                   
2006-12-16 18:09:00                4.464                  0.136   234.66   
2006-12-17 01:04:00                4.582                  0.258   238.08   
2006-12-17 01:08:00                4.618                  0.104   239.61   
2006-12-17 01:19:00                4.636                  0.140   237.37   
2006-12-17 01:20:00                4.634                  0.152   237.17   
...                                  ...                    ...      ...   
2010-11-24 07:53:00                4.666                  0.000   235.72   
2010-11-24 07:54:00                4.694                  0.000   236.78   
2010-11-24 07:55:00                4.602                  0.000   237.08   
2010-11-24 07:56:00                4.536                  0.000   237.03   
2010-11-24 07:57:00                4.626                  0.000   236.78   

           

In [20]:
sample, means = random_sample_means(df)
print(sample,means)


                     Global_active_power  Global_reactive_power  Voltage  \
Datetime                                                                   
2010-07-07 18:10:00                0.256                  0.106   242.00   
2007-05-14 06:50:00                0.466                  0.352   237.22   
2007-09-26 18:10:00                0.758                  0.194   238.66   
2007-06-19 07:30:00                1.290                  0.046   240.64   
2010-05-10 04:43:00                0.428                  0.202   242.23   
...                                  ...                    ...      ...   
2007-04-26 21:58:00                1.892                  0.174   232.74   
2007-11-11 18:00:00                1.098                  0.050   241.16   
2010-05-17 23:04:00                0.306                  0.000   246.83   
2008-08-12 18:35:00                0.252                  0.232   242.05   
2009-02-17 02:33:00                0.288                  0.052   245.05   

           

In [21]:
evening_df = evening_selection(df)
print(evening_df)

                     Global_active_power  Global_reactive_power  Voltage  \
Datetime                                                                   
2006-12-16 18:05:00                6.052                  0.192   232.93   
2006-12-16 18:08:00                6.308                  0.116   232.25   
2006-12-28 20:58:00                6.386                  0.374   236.63   
2006-12-28 21:02:00                8.088                  0.262   235.50   
2006-12-28 21:05:00                7.230                  0.152   235.22   
...                                  ...                    ...      ...   
2010-11-20 18:30:00                6.620                  0.314   229.00   
2010-11-20 18:34:00                6.266                  0.360   229.82   
2010-11-20 18:38:00                6.302                  0.376   229.65   
2010-11-20 18:42:00                6.238                  0.344   229.37   
2010-11-20 18:46:00                6.438                  0.308   229.26   

           

C:\Users\Ivan Moroz\AppData\Local\Temp\ipykernel_9352\2482100157.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['group1'] = subset['Sub_metering_1']  # пральна машина
C:\Users\Ivan Moroz\AppData\Local\Temp\ipykernel_9352\2482100157.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['group2'] = subset['Sub_metering_2']  # холодильник/освітлення
C:\Users\Ivan Moroz\AppData\Local\Temp\ipykernel_9352\2482100157.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [22]:
evening_df = normalize_and_standardize(evening_df, ['group1','group2','group3'])
print(evening_df)


                     Global_active_power  Global_reactive_power  Voltage  \
Datetime                                                                   
2006-12-16 18:05:00                6.052                  0.192   232.93   
2006-12-16 18:08:00                6.308                  0.116   232.25   
2006-12-28 20:58:00                6.386                  0.374   236.63   
2006-12-28 21:02:00                8.088                  0.262   235.50   
2006-12-28 21:05:00                7.230                  0.152   235.22   
...                                  ...                    ...      ...   
2010-11-20 18:30:00                6.620                  0.314   229.00   
2010-11-20 18:34:00                6.266                  0.360   229.82   
2010-11-20 18:38:00                6.302                  0.376   229.65   
2010-11-20 18:42:00                6.238                  0.344   229.37   
2010-11-20 18:46:00                6.438                  0.308   229.26   

           

In [23]:
pearson, spearman = correlation(df, 'Global_active_power', 'Voltage')
print(pearson,spearman)


-0.39976160962896157 -0.3271212377155308


In [24]:

df_encoded = pd.get_dummies(df, columns=['hour'], prefix='hour')
print(df_encoded.head())

                     Global_active_power  Global_reactive_power  Voltage  \
Datetime                                                                   
2006-12-16 17:24:00                4.216                  0.418   234.84   
2006-12-16 17:25:00                5.360                  0.436   233.63   
2006-12-16 17:26:00                5.374                  0.498   233.29   
2006-12-16 17:27:00                5.388                  0.502   233.74   
2006-12-16 17:28:00                3.666                  0.528   235.68   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
Datetime                                                                
2006-12-16 17:24:00              18.4             0.0             1.0   
2006-12-16 17:25:00              23.0             0.0             1.0   
2006-12-16 17:26:00              23.0             0.0             2.0   
2006-12-16 17:27:00              23.0             0.0             1.0   
2006-12-16 17:28:00          